In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

/Users/r3dmaohong/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

In [3]:
def conv2d(img, w, b):
    return( tf.nn.relu( tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1,1,1,1], padding='SAME'), b) ) )
def max_pool(img, k):
    return(tf.nn.max_pool(img, ksize = [1,k,k,1], strides=[1,k,k,1], padding='SAME'))

In [4]:
mnist = input_data.read_data_sets("../MNIST_data/", one_hot=True)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
learning_rate = 0.001
training_epochs = 5000
batch_size = 128
display_step = 100

In [6]:
n_input = 784
n_classes = 10
dropout = 0.5

In [7]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

_X = tf.reshape(x, shape=[-1,28,28,1])

In [8]:
# first layer
wc1 = tf.Variable(tf.random_normal([5,5,1,32])) # patch size, input channel, output channel
bc1 = tf.Variable(tf.random_normal([32])) # 32 feature maps?

conv1 = conv2d(_X, wc1, bc1)
conv1 = max_pool(conv1, k=2)

In [9]:
#dropout
keep_prob = tf.placeholder(tf.float32)
conv1 = tf.nn.dropout(conv1, keep_prob)

In [10]:
# second layer
wc2 = tf.Variable(tf.random_normal([5,5,32,64])) #
bc2 = tf.Variable(tf.random_normal([64]))

conv2 = conv2d(conv1, wc2, bc2)
conv2 = max_pool(conv2, k=2)

In [11]:
#dropout
conv2 = tf.nn.dropout(conv2, keep_prob)

In [12]:
# fully connected layer?
wd1 = tf.Variable(tf.random_normal([7*7*64, 1024])) # 28/2/2 = 7
bd1 = tf.Variable(tf.random_normal([1024]))

In [13]:
dense1 = tf.reshape(conv2, [-1, wd1.get_shape().as_list()[0]])
dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, wd1), bd1))
dense1 = tf.nn.dropout(dense1, keep_prob)

In [14]:
# output
wout = tf.Variable(tf.random_normal([1024, n_classes]))
bout = tf.Variable(tf.random_normal([n_classes]))

In [15]:
pred = tf.add(tf.matmul(dense1, wout), bout)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

In [16]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [17]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [18]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size=batch_size)
        sess.run(optimizer, feed_dict={x:batch_xs, y:batch_ys, keep_prob:dropout})
        
        if ((epoch+1) % display_step==0) | (epoch==0):
            acc = sess.run(accuracy, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.})
            loss = sess.run(cost, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.})
            print("Epoch " + "%05d"%(epoch+1) + ", Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
    print("="*50)
    print("Finished.")
    print("="*50)
    print("Testing Accuracy:", \
        accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.}))

Epoch 00001, Loss= 83024.718750, Training Accuracy= 0.18750
Epoch 00100, Loss= 9485.380859, Training Accuracy= 0.49219
Epoch 00200, Loss= 3739.992676, Training Accuracy= 0.60156
Epoch 00300, Loss= 1384.069824, Training Accuracy= 0.67969
Epoch 00400, Loss= 1261.957520, Training Accuracy= 0.68750
Epoch 00500, Loss= 1459.150757, Training Accuracy= 0.67188
Epoch 00600, Loss= 878.726501, Training Accuracy= 0.72656
Epoch 00700, Loss= 815.576721, Training Accuracy= 0.68750
Epoch 00800, Loss= 711.362427, Training Accuracy= 0.71094
Epoch 00900, Loss= 618.091919, Training Accuracy= 0.74219
Epoch 01000, Loss= 491.558533, Training Accuracy= 0.74219
Epoch 01100, Loss= 617.497192, Training Accuracy= 0.78125
Epoch 01200, Loss= 460.298309, Training Accuracy= 0.74219
Epoch 01300, Loss= 617.620850, Training Accuracy= 0.72656
Epoch 01400, Loss= 393.846008, Training Accuracy= 0.81250
Epoch 01500, Loss= 368.432404, Training Accuracy= 0.75000
Epoch 01600, Loss= 394.334778, Training Accuracy= 0.79688
Epoch 0